In [1]:
pip install torch transformers soundfile sounddevice numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!apt-get update && apt-get install -y portaudio19-dev python3-pyaudio
!pip install sounddevice soundfile transformers torch numpy pyaudio

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,773 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,241 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restrict

In [3]:
from IPython.display import Audio, display
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from transformers import pipeline
import numpy as np
from datetime import datetime
import IPython.display as ipd
from google.colab import files

class WhisperTranscriberColab:
    def __init__(self, model_size="base"):
        """
        Initialize the Whisper model
        model_size options: "tiny", "base", "small", "medium", "large"
        """
        print(f"Loading Whisper {model_size} model...")
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_size = model_size
        self.sample_rate = 16000

        # Load model and processor
        model_name = f"openai/whisper-{model_size}"
        self.processor = WhisperProcessor.from_pretrained(model_name)
        self.model = WhisperForConditionalGeneration.from_pretrained(model_name).to(self.device)

        # Alternative: Use pipeline for simpler usage
        self.pipe = pipeline(
            "automatic-speech-recognition",
            model=model_name,
            device=self.device
        )

        print(f"Model loaded successfully! Using device: {self.device}")

    def transcribe_uploaded_file(self, audio_file):
        """
        Transcribe uploaded audio file
        """
        print("Processing audio...")

        # Use pipeline for file transcription
        result = self.pipe(audio_file)
        return result["text"]

def save_transcription(text, filename=None):
    """Save transcription to file"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"transcription_{timestamp}.txt"

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)
    return filename

def main():
    print("Whisper Speech Recognition System for Google Colab")
    print("===============================================")

    model_sizes = ["tiny", "base", "small", "medium", "large"]
    print("\nAvailable model sizes:", ", ".join(model_sizes))
    print("Note: Larger models are more accurate but require more memory and processing power")

    model_size = input(f"\nChoose model size (default: base): ").strip().lower()
    if model_size not in model_sizes:
        model_size = "base"

    transcriber = WhisperTranscriberColab(model_size)

    print("\nPlease upload an audio file (supported formats: mp3, wav, m4a, etc.)")
    uploaded = files.upload()

    for filename in uploaded.keys():
        print(f"\nProcessing file: {filename}")
        try:
            # Transcribe
            transcription = transcriber.transcribe_uploaded_file(filename)

            print("\nTranscription:")
            print("--------------")
            print(transcription)

            transcript_file = save_transcription(transcription)
            print(f"\nTranscription saved to: {transcript_file}")

            files.download(transcript_file)

        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")

if __name__ == "__main__":
    main()

Whisper Speech Recognition System for Google Colab

Available model sizes: tiny, base, small, medium, large
Note: Larger models are more accurate but require more memory and processing power

Choose model size (default: base): medium
Loading Whisper medium model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

Saving test1.mp3 to test1 (1).mp3

Processing file: test1 (1).mp3
Processing audio...


/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.



Transcription:
--------------
 All right, now we will be running the pre-trained model from the OpenAI which is something called the WISPAR model.

Transcription saved to: transcription_20250402_061229.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>